## Marketing analitics

Исследование полей датасета

Расчет традиционных маркетиговых метрик

Классификация продаж по ценовым группам

Характеристики продаж за последний месяц

Причины ухода покупателей

### 1. Загрузка библиотек и данных

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
file ='Data Sample (AIC) для итоговой работы.xlsx'
xl   = pd.ExcelFile(file)
data = xl.parse('Online Retail')

In [ ]:
data = data.iloc[:,[4,6,7,0,1,3,5]]
data.head(1)

In [ ]:
data['InvSum'] = data.Quantity * data.UnitPrice
data.head(1)

### 2. Исследование полей датасета

#### 2.1. InvoiceDate


In [ ]:
data.InvoiceDate.describe()

In [ ]:
data['year_month']=data.InvoiceDate.apply(lambda x: str(x)[:7])
data['year'] =pd.to_datetime(data.InvoiceDate).dt.year
data['month']=pd.to_datetime(data.InvoiceDate).dt.month
data['week'] =pd.to_datetime(data.InvoiceDate).dt.week
data.head()

In [ ]:
data[data.year_month == '2011-12'].week.value_counts()

In [ ]:
data = data.iloc[:,[0,8,9,10,11,1,2,3,4,5,6,7]]
data.head(1)

Если анализ помесячно, то последний декабрь брать некорректно / половина месяца

In [ ]:
data = data[data.year_month != '2011-12']

#### 2.2. InvoiceNo

In [ ]:
data.InvoiceNo.value_counts().shape   # 14877
data.InvoiceNo.value_counts().head()

По полю пропусков нет, одному заказу может соответствовать несколько позиций в чеке, все параметры Инвойса в целом для строки с одним инвойсом совпадают

#### 2.3. CustomerID

In [ ]:
# Количество Покупателей без номера
data[data.CustomerID.isna()].InvoiceNo.unique().shape
# data.CustomerID.value_counts()

In [ ]:
# Количество Покупателей с номером / идентифициророванных / зарегистрированных
data.CustomerID.unique().shape

Предположим, что Покупатели по которым не указан номер,  это покупатели не зарегистрированные на сайте, не заполнившие Анкету (о которых мы ничего не знаем). Отследить повторное посещение / покупку  незарегистрированного Покупателя невозможно 

Предположим
Каждый (незарегистрированный  / без номера)  Покупатель в составе датасета  - уникален, то есть оформил 1 Инвойс
Присвоим незарегистрированным покупателям (другие) уникальные номера и закрепим за полем CustomerID признак уникальности покупателя

Дополним Датасет признаком факта регистрации Покупателя на сайте Customer_reg

In [ ]:
data.loc[data.CustomerID.isna(),  'Customer_reg'] = 0
data.loc[data.Customer_reg.isna(),'Customer_reg'] = 1

lcd = dict(zip(
                data[data.CustomerID.isna()].InvoiceNo.unique(), 
                list(range(0,1204))
            ))

data.loc[data.CustomerID.isna(),'CustomerID'] = data.loc[data.CustomerID.isna(),'InvoiceNo'].apply(lambda x: lcd[x])
data.CustomerID = data.CustomerID.astype(int)
data.head()

In [ ]:
data.CustomerID.unique().shape

#### 2.4.Country

In [ ]:
data.Country.value_counts().head()

Так как более 90 % продаж сосредоточено в 1 стране, выделить полноценную аналитику по любой из других стран не выйдет (недостаточно данных). Корректным представляется сравнение продаж внутри страны и зарубежом (в целом)

In [ ]:
data.loc[data.Country == 'United Kingdom', 'Sales_type']  = 'Domestic_sales'
data.loc[data.Country != 'United Kingdom', 'Sales_type']  = 'Foreign_sales'
data.Sales_type.value_counts()

#### 2.5. Quantity

In [ ]:
data.Quantity.describe()

In [ ]:
data [data.Quantity == 0].shape  # =0
data [data.Quantity  < 0].shape  # = 1083
data [data.Quantity>2000].shape  # = 2

data.Quantity.hist()

In [ ]:
data [data.Quantity  < -500]
# data [data.Quantity  > 1500]
data [data.StockCode == 23005].Quantity.value_counts()

Нет понимания, что делать с отрицательным количеством товара, которые никак логически не соотносятся ни с  количеством проданных товаров того же артикула по обной сделке, ни с общим количеством по артикулу за истории. Удалим записи с отрицательным количеством товара / до выяснения 


In [ ]:
data = data [data.Quantity  > 0]
data.describe()

#### 2.6. UnitPrice

In [ ]:
data [(data.UnitPrice  == 0)].head()

похоже на несостоявшиеся сделки или брошенные корзины, доля наблюдений незначительно, исключим записи из датасета

In [ ]:
data = data [data.UnitPrice  > 0]

### 3. Традиционные маркетинговые метрики

In [ ]:
m_metrics = pd.pivot_table(data,
                               index  = ['year_month'], 
                               values = ['InvoiceNo', 'StockCode', 'Quantity', 
                                         'CustomerID', 'InvSum'],
                               aggfunc= {'CustomerID': pd.Series.nunique, 
                                         'InvoiceNo' : pd.Series.nunique, 
                                         'StockCode' : len, 
                                         'Quantity'  : np.mean, 
                                         'InvSum'    : np.sum})
                                         
m_metrics = m_metrics.iloc[:,[0,2,4,3,1]]
m_metrics.columns = ['customers', 'invoices', 'stocks', 'quantity',  'summa']
m_metrics.round(1)

In [ ]:
m_metrics['In_to_Cu']  = m_metrics['invoices'] / m_metrics['customers'] # Среднее кол-во заказов на Покупателя 
m_metrics['St_to_In']  = m_metrics['stocks']   / m_metrics['invoices']  # Среднее кол-во товаров в заказе
m_metrics['Avg_price'] = m_metrics['summa']   /  m_metrics['quantity'] / m_metrics['stocks']  # Средняя цена товара
m_metrics['Cast_sum']  = m_metrics['summa']   /  m_metrics['customers'] # Средняя сумма потраченная Покупателем 
m_metrics['Inv_sum']   = m_metrics['summa']   /  m_metrics['invoices']  # Средняя сумма заказа 
m_metrics.round(1)


In [ ]:
m_metrics.hist(figsize=(15,10), bins=30)
plt.show()

### 4. Классификация продаж по ценовым категориям товаров

In [ ]:
group_up   = pd.cut(data['UnitPrice'],[0, 2, 5, 10, 20, 999999]) 
group_su   = pd.cut(data['InvSum'],   [0, 5, 10, 20, 40, 100, 200,999999]) 

PT_UnitPrice_Sum = pd.pivot_table( data, 
                                   values  =['InvSum'], 
                                   index   = group_up, 
                                   columns = group_su,
                                   aggfunc = np.sum,
                                   fill_value = 0) 
PT_UnitPrice_Sum.round(1)

In [ ]:
data.loc[data.UnitPrice  < 1.5,                        'UnitPrice_Group'] = '1_minimum_price'
data.loc[(data.UnitPrice >=1.5) &(data.UnitPrice < 2.5),'UnitPrice_Group'] = '2_low_price'
data.loc[(data.UnitPrice >=2.5) & (data.UnitPrice < 5), 'UnitPrice_Group'] = '3_middle_price'
data.loc[(data.UnitPrice >= 5) & (data.UnitPrice <10), 'UnitPrice_Group'] = '4_high_price'
data.loc[(data.UnitPrice >=10),                        'UnitPrice_Group'] = '5_maximum_price'

In [ ]:
data.UnitPrice_Group.hist()

In [ ]:
PT_UnitPrice_Sum = pd.pivot_table( data, 
                                   values  = ['InvSum'], 
                                   index   = ['UnitPrice_Group'], 
                                   columns = ['year', 'month'],
                                   aggfunc = np.sum,
                                   fill_value = 0,
                                   margins=True) 
PT_UnitPrice_Sum_percentage = np.round(PT_UnitPrice_Sum*100/PT_UnitPrice_Sum.iloc[-1, -1], 1)
PT_UnitPrice_Sum_percentage

### 5.  История активности Покупателей



In [ ]:
data['LTV']    = np.nan         # Сумма покупок за срок жизни клиента / здесь и далее по разделу - до даты наблюдения
data['LTCS']   = np.nan         # Количество покупок за срок жизни клиента - LifeTimeCountofSales
data['LTP']    = np.nan         # Количество месяцев жизни клиента - LifeTimePeriod
data['LTAOV']  = np.nan         # Средняя сумма покупки за срок  жизни клиента - LifeTimeAverageOrderValue
data['LTAUP']  = np.nan         # Средняя цена 1 товара за срок  жизни клиента - LifeTimeAverageUnitPrice
                                # медиана цен всех товаров, встретившихся в чеках (с учeтом 
                                # частоты встречаемости позиции в чеке, без учета доли в стоимости покупок)

In [ ]:
data = data.sort_values('InvoiceDate', ascending=[True])
data = data.reset_index().drop(['index'], axis = 1)

Расчет и классификация показателя LTV / Сумма всех покупок за срок жизни Покупателя

In [ ]:
for i in range(0,len(data)):
    data.iloc[i,15] = data.loc[ (data['CustomerID'] == data['CustomerID'][i])&    # LTV
                                (data['year_month'] <= data['year_month'][i]), 
                                'InvSum'].sum()                              
    
data.loc[(data.LTV > 0)   & (data.LTV <= 100),'LTVg'] = '0-100$'
data.loc[(data.LTV > 100) & (data.LTV <= 500),'LTVg'] = '100-500$'
data.loc[data.LTV >  500                     ,'LTVg'] = '>500$'
data.head()

Расчет и классификация показателя LTCS / Количество покупок за срок жизни Покупателя

In [ ]:
for i in range(0,len(data)):                            
    
    data.iloc[i,16] = data.loc[ (data['CustomerID'] == data['CustomerID'][i])&    # LTCS  
                                (data['year_month'] <= data['year_month'][i]), 
                                'InvoiceNo'].unique().shape[0]

Классификация показателя LTP / Срок жизни (активности) Покупателя в месяцах

In [ ]:
for i in range(0,len(data)):
    data.iloc[i,17] = data.loc[ (data['CustomerID'] == data['CustomerID'][i])&    # LTP
                                (data['year_month'] <= data['year_month'][i]), 
                                'year_month'].unique().shape[0]

data.loc[ data.LTP == 0,    'LTPg'] = '0'
data.loc[(data.LTP == 1),   'LTPg'] = '1_m'
data.loc[(data.LTP <  4)&
         (data.LTP >  1),   'LTPg'] = '2-3_m'
data.loc[(data.LTP >  3),   'LTPg'] = '4+_m'
data['LTPg'].value_counts()

Расчет / классификация показателя LTAOV / Средняя сумма покупки за срок жизни Покупателя

In [ ]:
data.loc[data['LTCS']  != 0, 'LTAOV'] = data['LTV']/data['LTCS']

data.loc[(data['LTCS'] != 0)&(data['LTAOV'] <20),                    'LTAOVg'] = '0-20$'
data.loc[(data['LTCS'] != 0)&(data['LTAOV'] >50),                    'LTAOVg'] = '50+ $'
data.loc[(data['LTCS'] != 0)&(data['LTAOV'] >20)&(data['LTAOV']<50), 'LTAOVg'] = '20-50$'
        
data['LTAOVg'].fillna('0$', inplace = True) 

PT_ASB = pd.pivot_table(data, 
                        values  = ['InvSum'], 
                        index   = ['LTAOVg'], 
                        columns = ['year', 'month'],
                        aggfunc = np.sum,
                        fill_value = 0,
                        margins=True) 
PT_ASB_percentage = np.round(PT_ASB*100/PT_ASB.iloc[-1, -1], 1)
PT_ASB_percentage

Расчет / классификация показателя LTAUP / Средняя (наиболее характерная) цена товара за срок жизни Покупателя

In [ ]:
for i in range(0,len(data)):
    data.iloc[i,19] = data.loc[ (data['CustomerID'] == data['CustomerID'][i])&    # LTP
                                (data['year_month'] <= data['year_month'][i]), 
                                'UnitPrice'].median()


In [ ]:
data.fillna(0, inplace = True) 

In [ ]:
def classify_unit_price (x):   
    z='0'
    if x > 0:
        z = '< 1$'
        if x > 1:
            z = '1-2$'
            if x > 2:
                z = '>2$'
    return z

data['LTAUPg'] = data['LTAUP'].astype(int).apply(classify_unit_price) 
data['LTAUPg'].value_counts()

### 6. Показатели покупок в  текущем месяце / месяце покупки

####  Сумма покупок за последний / текущий месяц

In [ ]:
data['LMTV'] = np.nan       # Last month time value - сумма покупок за последний месяц наблюдений
data['LMTVg'] = np.nan      # LMTV сгруппированные значения

In [ ]:
def classify_sum (x):
    z = 'no_sales'
    if x.InvSum.sum() > 0:
        z = '<50$'
        if x.InvSum.sum() > 50:
            z = '50-150$'
            if x.InvSum.sum() > 150:
                z = '>150$'
    return z
    
classify_rate_1 = data.groupby(['CustomerID', 'year_month']).apply(classify_sum)
classify_rate_2 = data.groupby(['CustomerID', 'year_month']).InvSum.sum()

In [ ]:
data['LMTV'] = np.nan
data['LMTVg'] = np.nan

for i in range(len(data)):
    data.iloc[i,25]  = classify_rate_1[data.iloc[i,5], data.iloc[i,1]]
    data.iloc[i,24]  = classify_rate_2[data.iloc[i,5], data.iloc[i,1]].round(1)
    
data['LMTVg'].value_counts()

####  Первая покупка на сайте (да/нет)

In [ ]:
data['First_sale'] = np.nan

for i in range(len(data)):
    if data.iloc[i,17]  == 1:
        if data.loc[(data['CustomerID'] == data['CustomerID'][i])& 
                    (data['year_month'] == data['year_month'][i]),
                    'InvSum'].sum() > 0:
            data.iloc[i,26] = 1
    else:
        data.iloc[i,26] = 0
    
data['First_sale'].value_counts()

### 7. Метрики ушедших покупателей

По состоянию на каждую дату, определим состав покупателей, которые прекратили совершать покупки по следующему алгоритму:
- определим в каких месяцах / периодах каждый из покупателей совершал покупки (сумма покупок >0) / Customer_sales_month;
- для каждого покупателя на каждый месяц определим значения:
а) если покупки были - присвоим '0' - клиент за месяц был активен / не уходил;
б) если покупок за месяц не было - присвоим значение количества месяцев,  прошедших с последней покупки до месяца наблюдения; / Customer_rate_month
- из полученных данных сформируем df в формате :  CustomerID/Месяц наблюдения/Месяц последней покупки/срок ... Покупателей прекративших активность более 3 месяцев назад исключаем из выборки, как (условно) не подлежащих возврату / Customers_away
- merge Customers_away и LifeTime показателями основного датасета и получаем базу для аналитики состава Покупателей, утративших лояльность, которая ляжет в основу Дашборда 

In [ ]:
Customer_sales_month = pd.pivot_table(data,
                       columns  = ['year_month'], 
                       index    = ['CustomerID'],
                       values   = ['InvSum'],
                       aggfunc  = 'sum',
                       fill_value = 0)

Customer_sales_month.tail()

In [ ]:
Customer_rate_month = pd.DataFrame(data = np.zeros((len(Customer_sales_month),len(Customer_sales_month.columns))))

periods  = ['2010-12', '2011-01','2011-02', '2011-03', '2011-04', '2011-05', 
            '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11']

Customer_rate_month.columns  = periods

z = 0
for i in range(0,len(Customer_sales_month.columns)): 
    for j in range (0,len(Customer_sales_month)): 
        if Customer_sales_month.iloc[j,i] > 0 or i ==0:
            z = 0            
        elif Customer_sales_month.iloc[j,i] ==0 and i >0:
            z = 1
            if Customer_sales_month.iloc[j,i-1] ==0 and i >1:
                z = 2
                if Customer_sales_month.iloc[j,i-2] ==0 and i >2:
                    z = 3
                    if Customer_sales_month.iloc[j,i-3] ==0 and i >3:
                        z = 4                                
        Customer_rate_month.iloc[j,i] = z     
        
Customer_rate_month.index = Customer_sales_month.index
Customer_rate_month.tail()

In [ ]:
Customers_away         = Customer_rate_month.iloc[:,[0]].reset_index().astype(int)
Customers_away.columns = ['CustomerID', 'months_away']
Customers_away['year_month']= '2010-12'
Customers_away['new_col']= 0

for i in range (1,12):
    z = Customer_rate_month.iloc[:,[i]].reset_index()
    z.columns = ['CustomerID', 'months_away']  
    z['year_month']= Customer_rate_month.columns[i]
    z['new_col']= i - z['months_away']
    Customers_away = pd.concat([Customers_away, z]) 

Customers_away.months_away = Customers_away.months_away.astype(int).round(0)
Customers_away.months_away.value_counts()

In [ ]:
Customers_away = Customers_away.reset_index().drop(['index'], axis = 1)
Customers_away = Customers_away.sort_values(['CustomerID', 'year_month'], ascending= True)
Customers_away['new_col'] = Customers_away['new_col'].astype(int)
Customers_away.tail(6)

In [ ]:
def classify_p (x):
    return periods[x]
Customers_away['New_year_month']= Customers_away.new_col.apply(classify_p)
Customers_away.tail()

In [ ]:
data_away = data.iloc[:,[1,5,12,13,16,17,18,19,20,21,22,23,24,25,26]].fillna(0)
data_away.head(1)

In [ ]:
Customers_away_info = Customers_away.merge(  data_away, 
                                             how = 'left', 
                                             left_on =['CustomerID','New_year_month'], 
                                             right_on=['CustomerID','year_month'])

Customers_away_info = Customers_away_info.drop_duplicates()
Customers_away_info.tail()

In [ ]:
Customers_away_info = Customers_away_info.iloc[:,[0,1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18]]

In [ ]:
Customers_away_info.rename(columns={'year_month_x': 'month_look', 'New_year_month': 'month_l_sale'}, inplace=True)

In [ ]:
Customers_away_info_filtered = Customers_away_info [(Customers_away_info.months_away  >  0)&
                                                    (Customers_away_info.months_away  <  4)&
                                                    (Customers_away_info.Customer_reg >= 0)&
                                                    (Customers_away_info.LMTV         > 0)
                                                    
                                                 ]
Customers_away_info_filtered.head()

In [ ]:
Customers_away_info_filtered_1 = Customers_away_info_filtered [Customers_away_info.months_away  ==1]
Customers_away_info_filtered_1.shape

In [ ]:
full_metrics = m_metrics.iloc[:,[0,4]]

away_metrics = Customers_away_info_filtered[
               Customers_away_info_filtered.months_away==1].iloc[:,[0,1,2,14]
              ].groupby(['month_look'])['months_away','LMTV'].sum()

basic_table  =  full_metrics.merge(away_metrics,
                                   how = 'left', 
                                   left_on =['year_month'], 
                                   right_on=away_metrics.index)
basic_table.columns = ['Period', 'Customers', 'Sales_gross', 'Customers_away', 'Sales_customers_away']
basic_table['Customers_away_%'] = 100*basic_table['Customers_away'] / basic_table['Customers']
basic_table['Sales_away_%']     = 100*basic_table['Sales_customers_away'] / basic_table['Sales_gross']
bt1 = basic_table.set_index('Period').iloc[1:].round(1)
bt2 = bt1.iloc[:,2:]
bt2

In [ ]:
writer = pd.ExcelWriter('DS LC.xlsx', engine='xlsxwriter')
   
data.to_excel(                          writer, 'dataset')
m_metrics.to_excel(                     writer, 'm_metrics')
Customers_away_info_filtered.to_excel(  writer, 'Lost_customers')
Customers_away_info_filtered_1.to_excel(writer, 'Lost_customers_filt')
bt1.to_excel(                           writer, 'bt1')
bt2.to_excel(                           writer, 'bt2')
writer.save()